# Naive Bayes Classifier - From Scratch

## David Barranquero

In [1]:
import numpy as np
import pandas as pd
from math import log

In this task, we will implement a Naive Bayes Classifier from scratch. The Naive Bayes Classifier utilises Bayes Theorem, 

$$ P(Y|X_1,..., X_n) = \frac{P(X_1,..., X_n|Y) \times P(Y)}{P(X_1,...,X_n)}$$ for class Y, and features $X_1,...,X_n$.

From this, we attempt to calculate the posterior probability of a certain class, given the data observed. However, the theorem requires use to know the joint distributions of the features, which is generally not possible with real data. If however, we multiply by the constant denominator, and naively assume each feature is conditionally independent within each class, then we can significantly reduce the computational cost, and calculate a value which is still proportional to the posterior probability, allowing us to still accurately classify data. Our equation hence becomes,

$$P(Y|X_1,..., X_n) \propto P(X_1|Y) \times...P(X_n|Y)\times P(Y)$$

The idea is to perform this calculation for each class $y \in Y$, and then assign the class label to the class which has the highest 'proportional' posterior probability.

In [2]:
random_no = 4760396
np.random.seed(random_no)

The dataset is a dataset from the 1999 KDD Cup, which tasked participants with correctly classifiying network connections given a set of predictors and identifying intrusions. For this project, we will just be using the 10% dataset. The dataset and supporting documentation can be found at http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [3]:
columns = [
    'duration',
    'protocol_type',
    'service',
    'flag',
    'src_bytes',
    'dst_bytes',
    'land',
    'wrong_fragment',
    'urgent',
    'hot',
    'num_failed_logins',
    'logged_in',
    'num_compromised',
    'root_shell',
    'su_attempted',
    'num_root',
    'num_file_creations',
    'num_shells',
    'num_access_files',
    'num_outbound_cmds',
    'is_host_login',
    'is_guest_login',
    'count',
    'srv_count',
    'serror_rate',
    'srv_serror_rate',
    'rerror_rate',
    'srv_rerror_rate',
    'same_srv_rate',
    'diff_srv_rate',
    'srv_diff_host_rate',
    'dst_host_count',
    'dst_host_srv_count',
    'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate',
    'dst_host_srv_serror_rate',
    'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate',
    'attack_type'
]

We begin first by reading in the file and setting the columns.

In [4]:
file = pd.read_csv('NBData/kddcup.data_10_percent', header=None)
kddcup = pd.DataFrame(file)
kddcup.columns = columns

Next we display some information on our dataset

In [5]:
kddcup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494021 entries, 0 to 494020
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     494021 non-null  int64  
 1   protocol_type                494021 non-null  object 
 2   service                      494021 non-null  object 
 3   flag                         494021 non-null  object 
 4   src_bytes                    494021 non-null  int64  
 5   dst_bytes                    494021 non-null  int64  
 6   land                         494021 non-null  int64  
 7   wrong_fragment               494021 non-null  int64  
 8   urgent                       494021 non-null  int64  
 9   hot                          494021 non-null  int64  
 10  num_failed_logins            494021 non-null  int64  
 11  logged_in                    494021 non-null  int64  
 12  num_compromised              494021 non-null  int64  
 13 

We can see that we do not have any missing values, and all datatypes are quantitative, either discrete counts or continuous, except for our target variable. Furthermore, we display the first few rows, checking our input.

In [6]:
kddcup.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


We need to reduce the number of classes down from 23 to 5. We will hence create a mapping based on the information in the 'training_attack_types.txt' supporting file, for these 5 classes. The text can be found at http://kdd.ics.uci.edu/databases/kddcup99/training_attack_types

In [7]:
class_dict = {
    'back.': 'dos',
    'buffer_overflow.': 'u2r',
    'ftp_write.': 'r2l',
    'guess_passwd.': 'r2l',
    'imap.': 'r2l',
    'ipsweep.': 'probe',
    'land.': 'dos',
    'loadmodule.': 'u2r',
    'multihop.': 'r2l',
    'neptune.': 'dos',
    'nmap.': 'probe',
    'normal.': 'normal',
    'perl.': 'u2r',
    'phf.': 'r2l',
    'pod.': 'dos',
    'portsweep.': 'probe',
    'rootkit.': 'u2r',
    'satan.': 'probe',
    'smurf.': 'dos',
    'spy.': 'r2l',
    'teardrop.': 'dos',
    'warezclient.': 'r2l',
    'warezmaster.': 'r2l',
}

Then we use this dictionary to map to the five classes.

In [8]:
# Use a map to fill the values
kddcup['class'] = kddcup['attack_type'].map(class_dict)

# Check data worked
print(kddcup['class'].value_counts())

dos       391458
normal     97278
probe       4107
r2l         1126
u2r           52
Name: class, dtype: int64


We can also see from this that there is a major class imbalance, which may affect the performance of our classifier. Finally, we drop the old class column from the dataset.

In [9]:
kddcup = kddcup.drop('attack_type', axis=1)

### Handling Continuous Features

To handle our continuous features, we are going to use binning. We display the information on our continuous features.

In [10]:
kddcup_float = kddcup.select_dtypes('float64')
float_cols = kddcup_float.columns.tolist()
kddcup_float.describe()

,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000
mean,0.176687,0.176609,0.057433,0.057719,0.791547,0.020982,0.028997,0.753780,0.030906,0.601935,0.006684,0.176754,0.176443,0.058118,0.057412
std,0.380717,0.381017,0.231623,0.232147,0.388189,0.082205,0.142397,0.410781,0.109259,0.481309,0.042133,0.380593,0.380919,0.230590,0.230140
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.410000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


From this, we can see that our continuous features are all contained within the interval of (0,1). Furthermore, we have a very non-uniform dispersion of values. We will use a simple approach and cut the continuous values into 2 equal-width bins, hence placing all values <= 0.5 in one bin, and all values > 0.5 in another.

In [11]:
# Our new DataFrame
kddcup_binned = pd.DataFrame()

feature_columns = list(kddcup.columns)

In [12]:
# For loop to bin our continous variable
for column in feature_columns:

    if column in float_cols:
        kddcup_binned[column] = pd.cut(kddcup[column], 
                                       bins=2, 
                                       labels=['<=0.5', '>0.5'])
    
    else:
        kddcup_binned[column] = kddcup[column]

To store our probabilities, we will use a nested dictionary with the architecture: dictionary -> class -> feature -> value -> probability. 

In [13]:
probability_dict = {
    'dos': {},
    'normal': {},
    'probe': {},
    'r2l': {},
    'u2r': {}
}

# Remove the class label, on the end of the list
feature_columns.pop(-1)

'class'

### Handling Zero Counts

Next, we loop through the entire dataset and build the dictionary (only storing the initial small value) so that it has seen every possible value. This way, if we are unlucky enough to have unique class values in our test data, nothing breaks.

This small value is a very small value we are adding to all probabilities to fix the 'Zero Count' problem. This occurs when the count of a particular feature value for a given class is 0, and hence the probability P(X1|Y) = 0. As such, our entire multiplication becomes 0. The log transformation used to fix numerical underflow (explained in more detail later), also will have issues as log(0) does not exist. As such, we use Laplace smoothing, which involves adding a value (usually 1) to all the counts, and if the counts are large enough, this resolves our issue without affecting too much the probabilities. Because we are storing probabilities and not counts, we will use the value 0.000001.

In [14]:
# For every column
for column in feature_columns:
    
    # For every class I have, add this column to its nested dictionary
    for target_class in probability_dict:
        probability_dict[target_class][column] = {}

    # Then get all the possible column values
    all_column_values = set(kddcup_binned[column].tolist())

    # And for each of these
    for value in all_column_values:
        
        # For every class dictionary where this value appears, 
        # Add our 'Zero Count' fix
        for target_class in probability_dict:
            probability_dict[target_class][column][value] = 0.000001

### Training and Test Split

Now that we have initialised our dictionary, we can safely split the dataset into a training and test set, with 70% of our data in the training set, and 30% in the testing set. We will perform stratified sampling via proportional allocation, to ensure a proportional class distribution in both our training and testing set, which lowers the sampling variance, and should hopefully produce a better classifier.

In [15]:
# First we separate all the values of the data
dos_idx = kddcup_binned.index[kddcup['class'] == 'dos'].tolist()
normal_idx = kddcup_binned.index[kddcup['class'] == 'normal'].tolist()
probe_idx = kddcup_binned.index[kddcup['class'] == 'probe'].tolist()
r2l_idx = kddcup_binned.index[kddcup['class'] == 'r2l'].tolist()
u2r_idx = kddcup_binned.index[kddcup['class'] == 'u2r'].tolist()

# And store these in a list
classes = [dos_idx, normal_idx, probe_idx, r2l_idx, u2r_idx]

# Our function to split the records into training and test,
def split_train_test(test_split):
    training_idx_full = []
    test_idx_full = []
    
    # We perform this random sampling for each stratum, labels of the target variable
    for j in range(0, 5):  
        training_idx, test_idx = shuffle_and_split(classes[j], 
                                                   len(classes[j]), 
                                                   test_split)
        training_idx_full.extend(training_idx)
        test_idx_full.extend(test_idx)

    return kddcup_binned.iloc[training_idx_full], kddcup_binned.iloc[test_idx_full]


# Our function to decide which rows will be for the test data set
def shuffle_and_split(indices, length, test_ratio):
    np.random.shuffle(indices)
    test_set_size = round(length * test_ratio)
    test_indices = indices[:test_set_size]
    train_indices = indices[test_set_size:]
    return train_indices, test_indices


# We call our function to get the training and test data
training_data, test_data = split_train_test(0.3)


Next, we perform checks on our training and testing datasets to ensure they were split correcty.

In [16]:
# Sanity checks on data size
print('\nProportion of data in Training set:', round(len(training_data) / len(kddcup), 4))
print('Proportion of data in Test set:', round(len(test_data) / len(kddcup), 4))


Proportion of data in Training set: 0.7
Proportion of data in Test set: 0.3


In [17]:
# Calculating how well our Stratified Sampler performed
training_prop = training_data['class'].value_counts() / len(training_data)
test_prop = test_data['class'].value_counts() / len(test_data)
original_prop = kddcup['class'].value_counts() / len(kddcup)

# Build a DF containing the performance stats
strat_stats = pd.DataFrame({
    "Training Strat": training_prop,
    "Test Strat": test_prop,
    "Original DF": original_prop,
    "Training Diff": training_prop - original_prop,
    "Test Diff": test_prop - original_prop
}).sort_index()

# Display the stats
print(strat_stats.head(10))

        Training Strat  Test Strat  Original DF  Training Diff     Test Diff
dos           0.792392    0.792390     0.792391   4.692757e-07 -1.094980e-06
normal        0.196912    0.196908     0.196911   9.858647e-07 -2.300358e-06
probe         0.008314    0.008313     0.008313   2.819599e-07 -6.579084e-07
r2l           0.002279    0.002281     0.002279  -5.803212e-07  1.354087e-06
u2r           0.000104    0.000108     0.000105  -1.156779e-06  2.699159e-06


As we can see, we have maintained our class proportions in the training and testing set. Next, we will build our probability dictionary, using the data in the training set.

In [18]:
# First we divide our training data into the classes
class_dos = training_data[training_data['class'] == 'dos']
class_normal = training_data[training_data['class'] == 'normal']
class_probe = training_data[training_data['class'] == 'probe']
class_r2l = training_data[training_data['class'] == 'r2l']
class_u2r = training_data[training_data['class'] == 'u2r']

# For every column we have
for column in feature_columns:

    # For every class we have
    for class_df in class_dos, class_normal, class_probe, class_r2l, class_u2r:

        # Get the unique counts of each feature value, given that class
        unique_value_counts = class_df[column].value_counts()

        # And their names, for the dictionary keys
        unique_value_names = unique_value_counts.index.tolist()

        # And convert the counts to probabilities
        unique_value_probs = unique_value_counts.to_numpy() * (1/class_df.shape[0])

        # Then use an if else statement to add that particular feature values probability
        # To the previously initialised epsilon value
        if class_df is class_dos:
            for i in range(len(unique_value_names)):
                probability_dict['dos'][column][unique_value_names[i]] += unique_value_probs[i]

        elif class_df is class_normal:
            for i in range(len(unique_value_names)):
                probability_dict['normal'][column][unique_value_names[i]] += unique_value_probs[i]

        elif class_df is class_probe:
            for i in range(len(unique_value_names)):
                probability_dict['probe'][column][unique_value_names[i]] += unique_value_probs[i]

        elif class_df is class_r2l:
            for i in range(len(unique_value_names)):
                probability_dict['r2l'][column][unique_value_names[i]] += unique_value_probs[i]

        else:
            for i in range(len(unique_value_names)):
                probability_dict['u2r'][column][unique_value_names[i]] += unique_value_probs[i]


Before we begin testing, we will need to calculate our prior class probabilities.

In [19]:
prior_prob_dos = class_dos.shape[0] / training_data.shape[0]
prior_prob_normal = class_normal.shape[0] / training_data.shape[0]
prior_prob_probe = class_probe.shape[0] / training_data.shape[0]
prior_prob_r2l = class_r2l.shape[0] / training_data.shape[0]
prior_prob_u2r = class_u2r.shape[0] / training_data.shape[0]

### Handling Numerical Underflow

Now we can begin testing. When calculating the proportional posterior probability, we are going to address another issue that can occur with the Naive Bayes classifier, numerical underflow. Because we are multiplying many small values together which are less than 0, their product will get smaller and smaller. This can lead to the final value being rounded to 0 by the program, even though the true value is not 0. Hence, we cannot do comparison between posterior probabilities. To resolve this issue we can perform a log transformation of our values, as log(A x B) = log(A) + log(B), which will avoid accidentally rounding to 0. Furthermore, log transformations preserve the proportionality of the equation, meaning the A value and B value that maximise A x B will also maximise log(A) + log(B).

In [20]:
# We store our predictions in a list, to be used for the confusion matrix later
pred_list = []

# For every training record we have
for idx in range(0, test_data.shape[0]):

    # Retrieve the row
    test_record = test_data.iloc[idx]

    # We initialise the probabilities to 0
    dos_prob = 0
    normal_prob = 0
    probe_prob = 0
    r2l_prob = 0
    u2r_prob = 0

    # For every feature column we have for this test record
    for i in range(len(feature_columns)):
        
        # Add the log of the conditional probability to the probability
        dos_prob += log(probability_dict['dos'][columns[i]][test_record.iloc[i]])
        
        normal_prob += log(probability_dict['normal'][columns[i]][test_record.iloc[i]])
        
        probe_prob += log(probability_dict['probe'][columns[i]][test_record.iloc[i]])
        
        r2l_prob += log(probability_dict['r2l'][columns[i]][test_record.iloc[i]])
        
        u2r_prob += log(probability_dict['u2r'][columns[i]][test_record.iloc[i]])

    # Then add the log of the prior probability
    dos_prob += log(prior_prob_dos)
    normal_prob += log(prior_prob_normal)
    probe_prob += log(prior_prob_probe)
    r2l_prob += log(prior_prob_r2l)
    u2r_prob += log(prior_prob_u2r)

    # Work out which probability if the max
    max_prob = max(dos_prob, normal_prob, probe_prob, r2l_prob, u2r_prob)

    # Whichever class maximised the proportional posterior probability, 
    # Is the class we predict.
    if max_prob == dos_prob:
        pred_list.append([test_record.name, 'dos'])
    elif max_prob == normal_prob:
        pred_list.append([test_record.name, 'normal'])
    elif max_prob == probe_prob:
        pred_list.append([test_record.name, 'probe'])
    elif max_prob == r2l_prob:
        pred_list.append([test_record.name, 'r2l'])
    elif max_prob == u2r_prob:
        pred_list.append([test_record.name, 'u2r'])


### Results

We will now manually construct a confusion matrix to display the classification results and assess our Naive Bayes Classifier's performance on the test dataset.

In [21]:
# Make a DataFrame from the predictions, attaching the row index (for joining)
pred_df = pd.DataFrame(pred_list, columns=['Row Index', 'Prediction'])

# And get the true class labels of our values
actual_df = test_data['class'].copy()

# Then we construct new dataframes, merging our pred & actual label for each record
performance_df = pd.merge(pred_df, actual_df, left_on='Row Index', right_index=True)

In [22]:
# Our function to construct a row of the confusion matrix
def confusion_matrix_row(label, DF, final_row):
    class_labels = ['dos', 'normal', 'probe', 'r2l', 'u2r']

    # Get the prediction counts for this class
    pred_counts = DF[DF['class'] == label]['Prediction'].value_counts()
    pred_counts_list = []
    total = 0
    sensitivity = 0
    
    # For every label in the class labels
    for j in range(len(class_labels)):  
        
        # If this label was predicted
        if class_labels[j] in pred_counts.index:  
            
            # Add the number of predictions to the list
            pred_counts_list.append(pred_counts.loc[class_labels[j]])  
            
            # Increment the counter
            total += pred_counts.loc[class_labels[j]]  
            
            # If this is on the diagonal, update the sensitivity
            if class_labels[j] == label:  
                sensitivity += pred_counts.loc[class_labels[j]]  
                
            # And the counter of the final row
            final_row[class_labels[j]] += pred_counts.loc[class_labels[j]]  
            
        # If we didn't see this class, we didn't label a test record as it, 
        # So assign a 0 to the row 
        else:  
            pred_counts_list.append(0)
            
    # Add the total of the true class size and the sensitivity        
    pred_counts_list.append(total)  

    # If the total is 0, just add 0, if not, add the sensitivity
    if total == 0:
        pred_counts_list.append(0)
    else:
        pred_counts_list.append(100 * sensitivity / total)
    return pred_counts_list, final_row

In [23]:
# Our function to construct the confusion matrices
def construct_confusion_matrix(dataframe):
    column_labels = ['dos', 
                     'normal', 
                     'probe', 
                     'r2l', 
                     'u2r', 
                     'Total', 
                     'Sensitivity']
    
    row_labels = ['dos', 
                  'normal', 
                  'probe', 
                  'r2l', 
                  'u2r', 
                  'Total']

    final_matrix_row = {
        'dos': 0,
        'normal': 0,
        'probe': 0,
        'r2l': 0,
        'u2r': 0
    }

    # We build one row a time, for all 'actual' classes, what were they predicted as
    row_dos, final_matrix_row = confusion_matrix_row('dos', 
                                                     dataframe, 
                                                     final_matrix_row)
    
    row_normal, final_matrix_row = confusion_matrix_row('normal', 
                                                        dataframe, 
                                                        final_matrix_row)
    
    row_probe, final_matrix_row = confusion_matrix_row('probe', 
                                                       dataframe, 
                                                       final_matrix_row)
    
    row_r2l, final_matrix_row = confusion_matrix_row('r2l', 
                                                     dataframe, 
                                                     final_matrix_row)
    
    row_u2r, final_matrix_row = confusion_matrix_row('u2r', 
                                                     dataframe, 
                                                     final_matrix_row)

    # We then construct the final row which contains how many of this class were labelled as such
    # And also calculate the accuracy
    total = sum(final_matrix_row.values())
    final_matrix_row['total'] = total
    final_matrix_row['sensitivity'] = (row_dos[0] +
                                       row_normal[1] +
                                       row_probe[2] +
                                       row_r2l[3] +
                                       row_u2r[4]) * 100 / total

    final_row = final_matrix_row.values()

    # We now construct our confusion matrix as a dataframe
    conf_matrix = pd.DataFrame(
        [row_dos,
         row_normal,
         row_probe,
         row_r2l,
         row_u2r,
         final_row],
        index=row_labels,
        columns=column_labels)

    # And return our matrix and accuracy
    accuracy = final_matrix_row['sensitivity']

    return conf_matrix, accuracy

In [24]:
# We construct a confusion matrix to visualise the models performance.
confusion_matrix, test_accuracy = construct_confusion_matrix(performance_df)
print('\nNaive Bayes Confusion Matrix')
print('\nActual/Predicted')
print(confusion_matrix.head(6))
print('\nNaive Bayes Test Accuracy:', test_accuracy)


Naive Bayes Confusion Matrix

Actual/Predicted
           dos  normal  probe  r2l  u2r   Total  Sensitivity
dos     113984     252   3201    0    0  117437    97.059700
normal       1   28046   1110   24    2   29183    96.103896
probe        3      29   1200    0    0    1232    97.402597
r2l          3       7      0  328    0     338    97.041420
u2r          0       4      0    1   11      16    68.750000
Total   113991   28338   5511  353   13  148206    96.871247

Naive Bayes Test Accuracy: 96.87124677813314


Overall, we achieved a test accuracy of roughly 97% which is an extremely strong classification performance. It is unsurprising that our 'dos' class had the highest sensitivity, and our 'u2r' class had our lowest sensitivity, given that 'dos' constituted roughly 80% of the dataset, and 'u2r' only 0.0001% of our dataset. Given this, 69% sensitivity is actually pretty good. In summary, our Naive Bayes classifier has performed remarkably well and achieved a very strong test accuracy, which makes it a very good classifier for classifying network connections, and identifying malicious connections.